<a href="https://colab.research.google.com/github/jazkre/01RAD/blob/main/R/01RAD_Ex09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 09


Today's exercise
 * Recap of the HW from the last Exercise
 * Multivariable Regression models
 * Transformation of response variable
 * Residual Analysis and Posthoc Annalysis

In [ ]:
# get requirements for this exercise
list_of_packages <- c("tidyverse", "lattice", "pwr", "MASS",
                      "GGally","nortest","lmtest","car") # "colorspace","RColorBrewer","pracma","olsrr","leaps","ISLR"
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
######################
# check our settings
# getwd()
# print(.libPaths())
# print(sessionInfo())
print(version)

tibble::tibble(
  Package = names(installed.packages()[,3]),
  Version = unname(installed.packages()[,3])
)


##  Recap of the last HW


In [ ]:
cars04 <- read.table("https://raw.githubusercontent.com/francji1/01RAD/main/data/carsdata2.csv",header=TRUE,sep=";")
head(cars04)

In [ ]:
#  Data Wrangling
str(cars04)

In [ ]:
# Check NA
colSums(is.na(cars04))

In [ ]:
cars04b <- cars04 %>% 
       mutate(consumption = 100/((1.60934*((CityMPG+HwyMPG)/2)/3.7854)),
              type = case_when(
                      Sedan ==1 ~ "sedan",
                      Sports==1 ~ "sport",
                      SUV==1    ~ "suv",
                      Wagon==1|Minivan==1|Pickup==1 ~ "minivan") %>% as.factor,
              wheel_drive = case_when(
                      AWD == 1 ~ "AWD",
                      RWD == 1 ~ "RWD")) %>% 
        replace_na(list(wheel_drive = "FWD")) %>% 
        mutate(wheel_drive = as.factor(wheel_drive)) %>%
        dplyr::select(RetailPrice,type, consumption,wheel_drive,DealerCost,EngineSize,Cyl,HP,Weight,WheelBase,Len,Width)

In [ ]:
summary(cars04b)


In [ ]:
table(cars04b$Cyl)


In [ ]:
cars04b <- cars04b %>% 
        filter(Cyl %in% c(4,6,8)) %>% 
        mutate(Cyl = as.factor(Cyl)) %>%
        na.omit()
summary(cars04b)
table(cars04b$type)


In [ ]:
## Visual check of the dataset
cars04b %>% dplyr::select(where(is.numeric)) %>% ggpairs()
cars04b %>% dplyr::select(where(is.factor)) %>% ggpairs()


*Questions*
* Build regression model and investigate 
  * Dependence of Weight on Enginesize (simple regression)
  * Dependence of Weight on Enginesize + HP  (aditive multivariate regression)
  * Dependence of Weight on Enginesize + HP + their interaction (multivariate regression)


In [ ]:
# Select only required variables and add "intraction"
cars04b %>% 
        dplyr::select(Weight,EngineSize,HP) %>% 
        mutate(HP_EngineSize = HP*EngineSize) %>% 
        ggpairs()


vykreslime si čo potrebujeme (gg-pairs)

In [ ]:
m1 <- lm(Weight ~  EngineSize,    data = cars04b)
summary(m1)


In [ ]:
m2 <- lm(Weight ~  EngineSize+HP, data = cars04b)
summary(m2)


In [ ]:

m3 <- lm(Weight ~  EngineSize*HP, data = cars04b)
summary(m3)

m1 a m2 maju rovnaké štatistiky, model sa nezlepšil

m3 2 stupne volnosti ale lepšie štat.

In [ ]:
# Compare models by F-test
anova(m1)
anova(m1,m2)
anova(m2,m3)


otestujeme modely

f-stat 758 -> eng size premenna, je jedina, je tam.

---
porovnavame modely proti sebe, 1 model musí byť podmodel druhého (obsiahnutý v druhom) musia mať rovnaké prem (nie transformované)

---
Ako zistím, že jeden model odpovedá druhemu? m1 ⊂ m2 ⊂ m3 ~> podľa stupňov voľnosti



In [ ]:
# Compute F test manually
SSR3  <- sum((m3$fitted.values - mean(cars04b$Weight))^2)
SSR1  <- sum((m1$fitted.values - mean(cars04b$Weight))^2)
SSR3-SSR1

In [ ]:
RSS3  <- sum(m3$residuals^2)
RSS1  <- sum(m1$residuals^2)
RSS1- RSS3


In [ ]:
MSE<- sum(m3$residuals^2)/m3$df.residual
MSE == summary(m3)$sigma^2
F  <- (SSR3-SSR1)/(MSE*(m1$df.residual-m3$df.residual))
F

In [ ]:
pf(abs(F), m1$df.residual-m3$df.residual,m3$df.residual, lower.tail = FALSE)
anova(m1,m3)


dostavame rovnake hodnoty

In [ ]:
plot(Weight ~ EngineSize, data = cars04b, pch=20, col = "black", xaxs="i",yaxs="i")
abline(m1, col ="blue")

scatterplot: úmernosť lin reg, korel medzi prem. 

(vykresľujeme len namerane data)
Vidíme že nám rastie rozptyl okolo reg. priamky



In [ ]:
cars04_m1 <- cars04b %>% 
        mutate(fit_m1 = m1$fitted.values)
ggplot(cars04_m1, aes(EngineSize, Weight)) +
        geom_point() +
        stat_smooth(method = lm, se = FALSE) +
        geom_segment(aes(xend = EngineSize, yend = fit_m1), color = "red", size = 0.3)


In [ ]:

par(mfrow = c(2, 2))
plot(m1, which = 1, pch = 20, col = "blue4", lwd = 2)
plot(m1, which = 3, pch = 20, col = "blue4",  lwd = 2)
abline(lm(sqrt(abs(rstandard(m1)))~fitted.values(m1)), col = "blue", lwd = 2)
plot(cars04b$EngineSize, residuals(m1), pch = 20, col = "blue4",  xlab = "Engine Size", ylab = "Residuals",
     main = "Residuals vs Covariate")
lines(lowess(cars04b$EngineSize, residuals(m1)), col = "red3", lwd = 2)
plot(cars04b$EngineSize, sqrt(abs(rstandard(m1))), pch = 20, col = "blue4", bg = "skyblue",
     xlab = "Engine Size", ylab = as.expression(substitute(sqrt(abs(yL)), list(yL = as.name("Standardized residuals")))),
     main = "Scale-Location (Speed)")
lines(lowess(cars04b$EngineSize, sqrt(abs(rstandard(m1)))), col = "red3", lwd = 2)
abline(lm(sqrt(abs(rstandard(m1)))~cars04b$EngineSize), col = "blue", lwd = 2)


1. obrazok -> je v pohode? porušenie homoskedascity -> jeden z najčastejších problémov.

je tam outlier.

2. abs. h. -> rastúci trend (priamka - > sklonm)

In [ ]:


#Check Model Assumptions 

## Recap what assumption we have to test:

##(i)  Linearity and additivity of the relationship between dependent and independent variables:
#(a) The expected value of dependent variable is a straight-line function of each independent variable,
#     holding the others fixed.
#(b) The slope of that line does not depend on the values of the other variables.
#(c) The effects of different independent variables on the expected value of the dependent variable are additive.
##(ii) Statistical independence of the errors 
#    (in particular, no correlation between consecutive errors in the case of time series data)
##(iii) Homoscedasticity (constant variance) of the errors
#(a)  versus time (in the case of time series data)
#(b)  versus the predictions (fitted)
#(c)  versus any independent variable
##(iv) Normality of the error distribution. 


#Plot Residuals vs. fitted and vs. all covariates in all models
#Plot residual QQ plots to confirm/reject normality of residuals.
#Perform statistical hypothesis test to confirm/reject homoscedasticity, normality, independence of residuals.
#Is there any indication of heteroscedasticity?


In [ ]:
fig <- function(width, heigth){
 options(repr.plot.width = width, repr.plot.height = heigth)
 }
 options(jupyter.plot_scale=2)


m3:

In [ ]:
par(mfrow = c(3, 2))
plot(m3, which = 1, pch = 20, col = "blue4", lwd = 2)
plot(m3, which = 3, pch = 20, col = "blue4",  lwd = 2)
abline(lm(sqrt(abs(rstandard(m3)))~fitted.values(m3)), col = "blue", lwd = 2)
plot(cars04b$EngineSize, residuals(m3), pch = 20, col = "blue4",  xlab = "Engine Size", ylab = "Residuals",
     main = "Residuals vs Covariate")
lines(lowess(cars04b$EngineSize, residuals(m3)), col = "red3", lwd = 2)
plot(cars04b$EngineSize, sqrt(abs(rstandard(m3))), pch = 20, col = "blue4", bg = "skyblue",
     xlab = "Engine Size", ylab = as.expression(substitute(sqrt(abs(yL)), list(yL = as.name("Standardized residuals")))),
     main = "Scale-Location (Speed)")
lines(lowess(cars04b$EngineSize, sqrt(abs(rstandard(m3)))), col = "red3", lwd = 2)
abline(lm(sqrt(abs(rstandard(m3)))~cars04b$EngineSize), col = "blue", lwd = 2)
plot(cars04b$HP, residuals(m3), pch = 20, col = "blue4",  xlab = "HP", ylab = "Residuals",
     main = "Residuals vs Covariate")
lines(lowess(cars04b$HP, residuals(m3)), col = "red3", lwd = 2)
plot(cars04b$HP, sqrt(abs(rstandard(m3))), pch = 20, col = "blue4", bg = "skyblue",
     xlab = "HP", ylab = as.expression(substitute(sqrt(abs(yL)), list(yL = as.name("Standardized residuals")))),
     main = "Scale-Location (Speed)")
lines(lowess(cars04b$HP, sqrt(abs(rstandard(m3)))), col = "red3", lwd = 2)
abline(lm(sqrt(abs(rstandard(m3)))~cars04b$HP), col = "blue", lwd = 2)


1. stále rastie rozptyl

In [ ]:
par(mfrow = c(2, 2))
plot(m3)


tvar trumpeety?

In [ ]:

# Serial correlation in disturbances
# Time series correlation ??? 
par(mfrow = c(1, 1))
plot(resid(m3)[-length(resid(m3))],resid(m3)[-1],pch = 20, col = "blue",
     xlab = expression(r[i-1]), ylab = expression(r[i]),cex = 1.5) # no autocorelation
lines(lowess(resid(m3)[-length(resid(m3))],resid(m3)[-1]), col = "red3", lwd = 2)
abline(h = 0, col = "grey")
abline(lm(resid(m3)[-length(resid(m3))]~resid(m3)[-1]), col = "green", lwd = 2)


nedáva zmysel robiť seriovu korelaciu mame 1 značku či čo. ak niekto nezanezie chybu, tak není dovod to robiť? IDK

In [ ]:

# The Durbin-Watson test has the null hypothesis that the autocorrelation of the disturbances is 0. 
# from lmtest package
dwtest(m3, alternative = "two.sided")  # Two-sided  
# from car package
dwt(m3) 


In [ ]:

# Test for heteroskedasticity by Breusch-Pagan test - from library lmtest
#bptest fits a linear regression model to the residuals of a linear regression model 
bptest(m3,varformula = ~ fitted(m3))



In [ ]:
fig(30,10)
## Visual tests of normality - QQ plots
op <- par(mfrow=c(1,3))
plot(m3, which = 2, pch = 20, col = "blue4", lwd = 2)
qqPlot(m3, simulate=TRUE, line="quartiles")
plot(density(rstudent(m3)))
par(op)



In [ ]:

## Hypothesis tests of Homogeneity
lillie.test(residuals(m3))   # Lilliefors test
shapiro.test(resid(m3))      # Shapiro-Wilk test
ad.test(resid(m3))           # Anderson-Darling
#ols_test_normality(m3)

cor(cars04$EngineSize,cars04$HP)




porovnávame či majú rovnaké rozdelenia

2 z 3 normalitu zamietajú

už máme 500 pozorovaní, tj docela dosť

Čo s tým budeme robiť?



In [ ]:

### How to improve the model? 


In [ ]:


## We can see heavy tails and problems with larger events (heteroskedasticity)
## Regression with log-transformed response - no change in analysis
m3_log <- lm(log(Weight) ~  EngineSize*HP, data=cars04b)
summary(m3_log)
summary(resid(m3_log))
summary(resid(m3))


logaritmická reansf. ak vidím, že rozptyl rastie, tak nechceme aby rástol tak prudko (napr s kvadrátom a pod) 

lm(log(Weight) ~  EngineSize*HP, data=cars04b)

ako odozvy modelujeme log. váhy


máme teraz iné odhady -> to sa zmenilo.
ale zaujíma nás vzťah k Y (možnosť interpretácie) -> $log(y)= X \beta + \epsilon$ (aditívna chyba)

$Y= e^{X \beta + \epsilon}=e^{X \beta} \cdot e^\epsilon$  (multiplikatívna chyba)

Ak $X_i$ ->$X_i +1$
v prvom možeme 1 model odčítať od druhého a dostaneme o koľko sa to zväčší

V druhom to nemožeme. Mame 2 rce -> porovnávam to tak, že to podelím -> dostávam prechod $e^{\beta_1}$ a to je o koľko \% sa daná i-ta premenná zmenila

je to iná interpretácia.

táto transformácia je pre nás dôležitá. 

In [ ]:

summary(cars04b$Weight)
summary(log(cars04b$Weight))



In [ ]:

# compare models with and without log transform
op <- par(mfrow=c(2,2))
qqPlot(m3, simulate=TRUE, line="quartiles")
plot(density(rstudent(m3)))
qqPlot(m3_log, simulate=TRUE, line="quartiles")
plot(density(rstudent(m3_log)))
par(op)


In [ ]:

lillie.test(residuals(m3_log))   # Lilliefors test ->nezam
shapiro.test(resid(m3_log))      # Shapiro-Wilk test ->zam
ad.test(resid(m3_log))           # Anderson-Darling

#dev.off()
#graphics.off()



In [ ]:

# Checking constant error variance - homoskedasticity
# spreadLevelPlot- abbreviation slp(): used to assess potential heteroskedasticity
# indicattion of a spread related to the mean


In [ ]:
? spreadLevelPlot

In [ ]:
fig(30,20)
op <- par(mfrow=c(2,2))
plot(fitted(m3), rstudent(m3), col="blue")
abline(h=0, lty=2)
lines(lowess(fitted(m3), rstudent(m3)))
spreadLevelPlot(m3,col="blue")

plot(fitted(m3_log), rstudent(m3_log), col="black")
abline(h=0, lty=2)
lines(lowess(fitted(m3_log), rstudent(m3_log)))
spreadLevelPlot(m3_log, col="black")
par(op)

fig(.,.) -> mením rozmer obrázkov



spreadlevelplot: logaritmická škála -> viac sa vytiahnu tie problematické reziduá a automaticky nám radí aj transformáciu.

Is spread related to the mean? Does it indicate a violation of the assumptions (potential heteroskedasticity)?

In [ ]:
fig(10,10)
par(mfrow=c(1,1))
slp(lm(Weight ~  EngineSize*HP, data=cars04b))

In [ ]:
summary(m3_log2 <- powerTransform(Weight ~  EngineSize*HP, data=cars04b))


In [ ]:

###
m4 <- lm(Weight  ~EngineSize + HP + Cyl, data = cars04b)
m4
# Checking linearity
# Component-residual plot (Partial residual plots) - crPlots
crPlots(m4)

# Be careful , if X_i is highly correlated with any of the other regressor, 
# the variance indicated by the crPlots can be much less than the actual variance.


vidíme sklon

ako by sme čítali boxplot? pre factorovu premennu. zaujímajú nás stredné hodnoty v skupinách. či sa nejaká stredná hodnota diametrálne líši od ostatných. ukazuje mi, že je tam nejaký posun. je to dôležitá zmena. 

In [ ]:
termplot(m4, partial.resid=TRUE, terms="EngineSize")

In [ ]:

# Check residuals again,
# Resid against independent variables with residualPlots
residualPlots(m4)
# Absolute stud. Resid agains fitted  



In [ ]:
m3 <- lm(Weight ~  EngineSize, data = cars04b)
m2 <- lm(Weight ~  HP, data = cars04b)
m3 <- lm(Weight ~  EngineSize+HP, data = cars04b)
# Added Variable Plot - Partial regression plots 
avPlots(m1)
avPlots(m2)
avPlots(m3)

# avPlots can by used to:
# - identify data points with high leverage and influential data points
# - identify the nature of the relationship between Y and X_i
# - identify the relation without correlation effects
# - help to select variables



In [ ]:
#install.packages("effects")
library(effects)
# display effects


In [ ]:
plot(allEffects(m4))
op <- par(mfrow=c(1,1))


posledný obrázok -> conf intervaly sa pretínajú

Box-Cox with shifted values (case for negative response variable)

$$g(Y + a) = ln(Y + a) \ \text{if} \  \ λ = 0$$,
$$g(Y + a) = \frac{(Y + a)^{\lambda} - 1}{\lambda} \ \ \text{if} \   λ \neq 0$$,



voľba a - > posun doprava aby všetky boli kladné ale často je dobré pripustiť si aj nejaku rezervu 

In [ ]:
# Box-Cox transformation:
bc     <- boxcox(m1, lambda = seq(-1,1 , 1/100))
lambda <- bc$x[which.max(bc$y)]
lambda

λ=0.02 -> ak leží v CI nuly, tak berieme, že je to 0 -> ln() je stále interpretovateľný.

často transf pre $λ \neq 0$ nemáme fyzikálne pozadie a preto treba zvažovať prečo používame práve takú transf alebo inú.

In [ ]:
# Power transform
cars04c=cars04b
cars04c$Weight_power = (cars04c$Weight^lambda-1)/lambda
m3_bc = lm(Weight_power~EngineSize*HP, data=cars04c)
summary(m3_bc)


In [ ]:
fig(30,10)
op <- par(mfrow=c(2,3))
plot(m3,which=2)
plot(m3_bc,which=2)
plot(m3_log,which=2)
hist(resid(m3),breaks=30)
hist(resid(m3_bc),breaks=30)
hist(resid(m3_log),breaks=30)